<a href="https://colab.research.google.com/github/sreehari59/Polar-Embedding/blob/main/Polar_Embedding_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This is to load file from google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import gensim
from numpy import linalg
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath

# This is for the plotly subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# This is to download csv file
from google.colab import files

import random
import scipy


!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 211 µs (started: 2022-06-02 04:11:25 +00:00)


In [3]:
def unzipping_file():
  !wget https://nlp.stanford.edu/data/glove.6B.zip
  !unzip glove*.zip # When it is unzipped you can see the different set of dimension it can have that is 25d, 50d, 100d or 200d

time: 2.18 ms (started: 2022-06-02 04:11:25 +00:00)


In [4]:
def model_initialization(unzippedname, new_name):
  model_glove = glove2word2vec(unzippedname,new_name)
  model_glove = gensim.models.KeyedVectors.load_word2vec_format(new_name, binary=False)
  return model_glove

time: 3.28 ms (started: 2022-06-02 04:11:25 +00:00)


In [5]:
def generate_norm_embedding(model, output_path):
    temp_file = open(output_path,'wb')
    temp_file.write(str.encode(str(len(model.vocab))+' '+str(model.vector_size)+'\n'))
    
    for each_word in tqdm(model.vocab):
        temp_file.write(str.encode(each_word+' '))  
        temp_file.write(model[each_word]/linalg.norm(model[each_word]))
        temp_file.write(str.encode('\n'))
    
    
    temp_file.close()
    return output_path

time: 9.62 ms (started: 2022-06-02 04:11:25 +00:00)


In [6]:
def load_normalized_embedding(output_file_name):
  return gensim.models.KeyedVectors.load_word2vec_format(output_file_name,binary=True)

time: 1.65 ms (started: 2022-06-02 04:11:25 +00:00)


**Loading the Antonym**

In [7]:
def load_antonyms(list_new,current_model):
  list_new= list(dict.fromkeys(list_new).keys())

  similarity_matrix = defaultdict(list)
  for each_pair in tqdm(list_new):
      word1 = each_pair[0]
      word2 = each_pair[1]
      if word1 < word2:
          similarity_matrix[word1].append(word2)
      else:
          similarity_matrix[word2].append(word1)

  all_similarity = defaultdict(dict)
  for each_key in tqdm(similarity_matrix):
      for each_value in similarity_matrix[each_key]:
  #         cosine_similarity([current_model[each_key]]
          try:
            all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
          except:
            print("The word is not in the vocabulary : ",each_key," ",each_value)

  final_list = []
  for index_counter, each_key in enumerate(tqdm(all_similarity)):
  #     print(each_key,all_similarity[each_key])
      listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
  #     print(listofTuples)
      final_list.append((each_key, listofTuples[0][0]))
  print(len(final_list))

  list_antonym = final_list
  
  return list_antonym

time: 25.8 ms (started: 2022-06-02 04:11:25 +00:00)


**Find the antonym difference vectors**

In [8]:
def antonym_difference_vector(list_antonym,current_model):
  num_antonym  = len(list_antonym)
  antonymy_vector = []
  for each_word_pair in list_antonym:
      antonymy_vector.append(current_model[each_word_pair[0]]- current_model[each_word_pair[1]])
      
  antonymy_vector = np.array(antonymy_vector)
  print(antonymy_vector.shape)
  print(antonymy_vector)
  return antonymy_vector

time: 6.42 ms (started: 2022-06-02 04:11:25 +00:00)


In [9]:
def generate_embedding_path(current_model, embedding_path, binary, antonym_vector, curr_dim,antonymy_vector):
    curr_antonym_vector = antonymy_vector[antonym_vector[:curr_dim]]
    curr_antonymy_vector_inverse = np.linalg.pinv(np.transpose(curr_antonym_vector))
    new_embedding_dict = transform_to_antonym_space(current_model, embedding_path, binary,curr_antonymy_vector_inverse)

    return new_embedding_dict

time: 4.53 ms (started: 2022-06-02 04:11:25 +00:00)


In [10]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):

    temp_dict = dict()

    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    #print('New model size is',len(current_model), embedding_size)

    temp_file = None
    
    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)
        temp_dict[each_word] = new_vector
        
        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()
    return temp_dict

time: 27.5 ms (started: 2022-06-02 04:11:25 +00:00)


In [11]:
def make_polar_dict(company_name, antonym, embedding, top_n = False, n = 10):
  temp_dict = dict()
  temp_polar = embedding[company_name]

  if top_n:
    idx = np.argsort([abs(x) for x in temp_polar])[-n:]
    for i in idx:
      print(antonym[i],temp_polar[i],'\n')


  if len(antonym) == len(temp_polar):
    for a in range(len(antonym)):
      temp_dict[antonym[a]] = temp_polar[a]
    return temp_dict

time: 9.76 ms (started: 2022-06-02 04:11:25 +00:00)


In [12]:
def download_csv_file(df,filename):
  df.to_csv(filename)
  files.download(filename)

time: 1.68 ms (started: 2022-06-02 04:11:25 +00:00)


**The below function will input the antonym pair from the user**

In [13]:
def user_antonym_input(list_new,user_list):
  list_new.append(user_list)
  return list_new

time: 1.9 ms (started: 2022-06-02 04:11:25 +00:00)


In [15]:
def clear_tuple(tuples):
  temp = list(tuples)
  temp.clear()
  tuples = tuple(temp)
  return tuples

time: 2.95 ms (started: 2022-06-02 04:11:25 +00:00)


In [18]:
def random_word_list(number_of_elements):  
  list_antonym = []  
  with open('/content/drive/MyDrive/LenciBenotto.val') as fp:
      for line in fp:
          parts = line.split() # The line is split into 4 parts as there are 3 space in between
          if parts[3]=='antonym': # Here we if the the 4th part that is parts[3] in the line is antonym then that line is selected
              word1 = parts[0].split('-')[0]
              word2 = parts[1].split('-')[0]
              if word1 in current_model and word2 in current_model: # If both the words are there in the model then it is appended to the list
                  list_antonym.append((word1.strip().lower(), word2.strip().lower()))
  
  return random.sample(list_antonym, number_of_elements)

time: 10.9 ms (started: 2022-06-02 04:11:25 +00:00)


In [19]:
def similar_word_to_give_antonym_pair(input_list,current_model):
  new_similar_list= [] 

  for i in input_list:
    antonym_pair = i
    first_word = antonym_pair[0]
    second_word = antonym_pair[1]

    similar_word_list1 = current_model.similar_by_word(first_word)
    similar_word_list2 = current_model.similar_by_word(second_word)  
    
    new_antonym_word1 = similar_word_list1[0][0]
    new_antonym_word2 = similar_word_list2[0][0]   
   
    antonym_tuple = (new_antonym_word1,new_antonym_word2)
    
    new_similar_list.append(antonym_tuple)
    clear_tuple(antonym_tuple)
    
  return new_similar_list

time: 11.9 ms (started: 2022-06-02 04:11:25 +00:00)


**Below cell will load the model and unzip the model**

In [20]:
unzipping_file()
model_glove = model_initialization('glove.6B.300d.txt','gensim_glove.6B.300d.txt')
path = generate_norm_embedding(model_glove,'glove_norm_wiki.mod')
current_model  = load_normalized_embedding(path)

--2022-06-02 04:11:25--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-06-02 04:11:26--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.09MB/s    in 2m 44s  

2022-06-02 04:14:10 (5.00 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/400000 [00:00<?, ?it/s]

time: 4min 31s (started: 2022-06-02 04:11:25 +00:00)


**Random List**

In [62]:
list_powerdistance_random =[('make', 'break'), ('cameraman', 'playwright'), ('mystical', 'factual'), ('promotional', 'defamation'), ('iconic', 'unknown')]
list_individualism_random = [('lop', 'secure'), ('shah', 'poor'), ('pneumatic', 'solid'), ('interpret', 'misinterpret'), ('confer', 'refuse')]
list_masculinity_random = [('try', 'abstain'), ('fatalistic', 'freewill'), ('knowledgeable', 'uninformed'), ('confine', 'free'), ('fan', 'warm')]
list_longterm_random = [('innovator', 'follower'), ('sensory', 'numb'), ('hedge', 'squander'), ('arachnid', 'serpent'), ('disclose', 'secrete')]
list_indulgence_random = [('diagnose', 'sicken'), ('intercourse', 'disconnection'), ('sensory', 'sensorial'), ('emasculate', 'strengthen'), ('metropolitan', 'rural')]
list_uncertainity_avoidance_random = [('stretcher', 'compressor'), ('amalgamate', 'separate'), ('caretaker', 'assailant'), ('taker', 'violator'), ('contaminate', 'sterilize')]

time: 9.32 ms (started: 2022-06-02 05:24:50 +00:00)


**Human generated list**

In [63]:
list_powerdistance =[('hierarchical','non-hierarchical'),('superior','equal'),('leader','subordinate'),('inequality','equality'),('autocrat','democrat')]
list_individualism = [('individuality','community'),('self-interest','harmony'),('tasks','relationships'),('individual','groups'),('particularism','universalism')]
list_masculinity = [('achievement', 'support'),('competitive', 'caring'),('assertive', 'submissive'),('ambitious', 'unambitious'),('sucess','cooperation')]
list_longterm = [('pragmatic','normative'),('progress','preserve'),('adapt','conserve'),('developing','stable'),('advance','retain')]
list_indulgence = [('fulfillment','restriction'),('satisfaction','limitation'),('liberty','moderation'),('expand','direct'),('freedom','regulation')]
list_uncertainity_avoidance = [('clarity','complexity'),('clear','ambiguous'),('certain','uncertain'),('uniformity','diversity'),('agreement','variation')]


time: 9.78 ms (started: 2022-06-02 05:24:50 +00:00)


**The nearest pairs to Random list suggested by the pretrained model**

In [64]:
similar_list_powerdistance_random = similar_word_to_give_antonym_pair(list_powerdistance_random,current_model)
similar_list_individualism_random = similar_word_to_give_antonym_pair(list_individualism_random,current_model)
similar_list_masculinity_random = similar_word_to_give_antonym_pair(list_masculinity_random,current_model)
similar_list_longterm_random = similar_word_to_give_antonym_pair(list_longterm_random,current_model)
similar_list_indulgence_random = similar_word_to_give_antonym_pair(list_indulgence_random,current_model)
similar_list_uncertainity_avoidance_random = similar_word_to_give_antonym_pair(list_uncertainity_avoidance_random,current_model)

time: 3.05 s (started: 2022-06-02 05:24:50 +00:00)


**The nearest pairs to human made list suggested by the pretrained model**

In [65]:
similar_list_powerdistance = similar_word_to_give_antonym_pair(list_powerdistance,current_model)
similar_list_individualism = similar_word_to_give_antonym_pair(list_individualism,current_model)
similar_list_masculinity = similar_word_to_give_antonym_pair(list_masculinity,current_model)
similar_list_longterm = similar_word_to_give_antonym_pair(list_longterm,current_model)
similar_list_indulgence = similar_word_to_give_antonym_pair(list_indulgence,current_model)
similar_list_uncertainity_avoidance = similar_word_to_give_antonym_pair(list_uncertainity_avoidance,current_model)

time: 2.97 s (started: 2022-06-02 05:24:53 +00:00)


**In the below cell give the list to create the polar embedding**

In [66]:
list_new = list_masculinity

time: 2.07 ms (started: 2022-06-02 05:24:56 +00:00)


In [67]:
list_antonym  = load_antonyms(list_new,current_model)
len(list_new)
antonymy_vector = antonym_difference_vector(list_antonym,current_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/5 [00:00<?, ?it/s]

5
(5, 300)
[[ 0.14758141  0.09032236  0.07620825 ...  0.12463442  0.00482126
  -0.03302358]
 [-0.09652946 -0.07345466 -0.0032986  ...  0.12112679  0.01148236
  -0.0293146 ]
 [-0.05397214  0.02136535 -0.00466404 ... -0.04937481 -0.06371355
  -0.00237834]
 [ 0.03041583 -0.02709354  0.01676632 ... -0.02892689 -0.04289657
   0.17347242]
 [-0.05428931 -0.04361981 -0.01984703 ... -0.08699577  0.01813235
  -0.04817012]]
time: 237 ms (started: 2022-06-02 05:24:56 +00:00)


In [68]:
company = pd.read_csv('/content/drive/MyDrive/International_Fortune_GloVe.csv',encoding= 'unicode_escape')
name_list = company['0']

time: 15.2 ms (started: 2022-06-02 05:24:57 +00:00)


In [69]:
name_word_embedding = dict()
new_name_list=[]
for name in name_list:
  try:
    name_word_embedding[name] = current_model[name]
    new_name_list.append(name)
  except:
    print("The company is not there: ",name)

print(new_name_list)

The company is not there:  jpmorganchase
The company is not there:  generalmotors
The company is not there:  fanniemae
The company is not there:  bankofamerica
The company is not there:  nestlÃ©
The company is not there:  wellsfargo
The company is not there:  generalelectric
The company is not there:  sinopharm
The company is not there:  freddiemac
The company is not there:  waltdisney
The company is not there:  engie
The company is not there:  morganstanley
The company is not there:  indianoil
The company is not there:  telefÃ³nica
The company is not there:  bestbuy
The company is not there:  talanx
The company is not there:  abbvie
The company is not there:  americanexpress
The company is not there:  deutschebank
The company is not there:  tatamotors
The company is not there:  l'orÃ©al
The company is not there:  angloamerican
The company is not there:  achmea
The company is not there:  dollartree
The company is not there:  riteaid
['walmart', 'amazon', 'apple', 'volkswagen', 'mckesso

In [70]:
random_antonym_vector = [i for i in range(len(antonymy_vector))]
random.shuffle(random_antonym_vector)
print(len(random_antonym_vector))
name_new_embedding = generate_embedding_path(name_word_embedding, 'test_run',True ,random_antonym_vector,500,antonymy_vector)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


5


  0%|          | 0/142 [00:00<?, ?it/s]

time: 136 ms (started: 2022-06-02 05:24:57 +00:00)


In [71]:
antonym_500 = [list_antonym[x] for x in random_antonym_vector[:500]]

time: 2.45 ms (started: 2022-06-02 05:24:57 +00:00)


**Polar Embedding -> DataFrame**

In [72]:
df = dict()
for t in new_name_list:
  df[t] = make_polar_dict(t, antonym_500, name_new_embedding)

new_df = pd.DataFrame(df).transpose()


# change columns to better read names
new_columns = []

for pair in antonym_500:
  temp = pair[0]+''+pair[1]
  new_columns.append(temp)

new_df.columns = new_columns

time: 31.2 ms (started: 2022-06-02 05:24:57 +00:00)


**In the first run uncomment the cell below as the copy of new_df is made and it is into the concat_polar_embedding_df we concatinate the polar embedding**

In [73]:
concat_polar_embedding_df = new_df.copy()

time: 1.6 ms (started: 2022-06-02 05:24:57 +00:00)


**In the second run onwards comment the previous cell and uncomment the lower cell**

In [74]:
#concat_polar_embedding_df = concat_polar_embedding_df.join(new_df)

time: 1.05 ms (started: 2022-06-02 05:24:57 +00:00)


**Below cell is to download the file once it is ready**

In [75]:
'''
from google.colab import files
concat_polar_embedding_df.to_csv('concat_polar_embedding_df.csv',header=True)
files.download('concat_polar_embedding_df.csv')
'''

"\nfrom google.colab import files\nconcat_polar_embedding_df.to_csv('concat_polar_embedding_df.csv',header=True)\nfiles.download('concat_polar_embedding_df.csv')\n"

time: 8.76 ms (started: 2022-06-02 05:24:57 +00:00)
